![JohnSnowLabs](https://sparknlp.org/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp/blob/master/examples/python/transformers/openvino/HuggingFace_OpenVINO_in_Spark_NLP_Bart.ipynb)

# Import OpenVINO GPT2  models from HuggingFace 🤗 into Spark NLP 🚀

This notebook provides a detailed walkthrough on optimizing and exporting BGE models from HuggingFace for use in Spark NLP, leveraging the various tools provided in the [Intel OpenVINO toolkit](https://www.intel.com/content/www/us/en/developer/tools/openvino-toolkit/overview.html) ecosystem.

Let's keep in mind a few things before we start 😊

- OpenVINO support was introduced in  `Spark NLP 5.4.0`, enabling high performance inference for models. Please make sure you have upgraded to the latest Spark NLP release.


## 1. Export and Save the HuggingFace model

- Let's install `transformers` and `openvino` packages with other dependencies. You don't need `openvino` to be installed for Spark NLP, however, we need it to load and save models from HuggingFace.
- We lock `transformers` on version `4.48.3`. This doesn't mean it won't work with the future releases, but we wanted you to know which versions have been tested successfully.

In [ ]:
!pip install -q --upgrade transformers==4.48.3 optimum[openvino]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.6/70.6 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 16.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 348.7/348.7 kB 23.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 424.6/424.6 kB 28.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.5/10.5 MB 109.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 61.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 35.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 34.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.0 MB/s eta 0:00:0

[Optimum Intel](https://github.com/huggingface/optimum-intel?tab=readme-ov-file#openvino) is the interface between the Transformers library and the various model optimization and acceleration tools provided by Intel. HuggingFace models loaded with optimum-intel are automatically optimized for OpenVINO, while being compatible with the Transformers API.
- To load and export a Hugging Face model optimized for OpenVINO, we'll use the `!optimum-cli` command
- We'll use [openai-community/gpt2](https://huggingface.co/openai-community/gpt2) model from HuggingFace, representing an OpenVINO model.
- In addition the tokenizer and `sentencepiece.bpe.model` (if applicable) must be placed inside an `/assets` directory. These are required for tokenization within Spark NLP.

In [ ]:
MODEL_NAME = "openai-community/gpt2"
EXPORT_PATH = f"ov_models/{MODEL_NAME}"

!optimum-cli export openvino --model {MODEL_NAME} --task text-generation {EXPORT_PATH}

2025-07-03 09:18:38.305845: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1751534318.331161     977 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1751534318.338635     977 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-07-03 09:18:38.362354: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
Multiple distributions found for package optimum. Picked distribution: optimum
No CUDA runtime is found, using CUDA_HOME='/us

Create 'assets' directory required for Spark NLP compatibility

In [ ]:
!mkdir {EXPORT_PATH}/assets
!mv -t {EXPORT_PATH}/assets {EXPORT_PATH}/*.json {EXPORT_PATH}/*.txt

import json

output_json = json.load(open(f"{EXPORT_PATH}/assets/vocab.json"))
with open(f"{EXPORT_PATH}/assets/vocab.txt", "w") as f:
    for key in output_json.keys():
        print(key, file=f)

mkdir: cannot create directory ‘ov_models/openai-community/gpt2/assets’: File exists


In [ ]:
!ls -l {EXPORT_PATH}/assets

total 5120
-rw-r--r-- 1 root root     880 Jul  3 09:19 config.json
-rw-r--r-- 1 root root     119 Jul  3 09:19 generation_config.json
-rw-r--r-- 1 root root  456318 Jul  3 09:19 merges.txt
-rw-r--r-- 1 root root      99 Jul  3 09:19 special_tokens_map.json
-rw-r--r-- 1 root root     475 Jul  3 09:19 tokenizer_config.json
-rw-r--r-- 1 root root 3557680 Jul  3 09:19 tokenizer.json
-rw-r--r-- 1 root root  798156 Jul  3 09:19 vocab.json
-rw-r--r-- 1 root root  406992 Jul  3 09:19 vocab.txt


## Import and Save GPT2 in Spark NLP

- Install and set up Spark NLP in Google Colab
- This example uses specific versions of `pyspark` and `spark-nlp` that have been tested with the transformer model to ensure everything runs smoothly.

In [1]:
!pip install -q pyspark==3.5.4 spark-nlp==5.5.3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 4.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 635.7/635.7 kB 24.7 MB/s eta 0:00:00


Let's start Spark with Spark NLP included via our simple `start()` function

In [2]:
import sparknlp

spark = sparknlp.start()

print("Spark NLP version: {}".format(sparknlp.version()))
print("Apache Spark version: {}".format(spark.version))

Spark NLP version: 5.5.3
Apache Spark version: 3.5.4


- Let's use `loadSavedModel` functon in `GPT2Transformer` which allows us to load the Openvino model
- Most params will be set automatically. They can also be set later after loading the model in `GPT2Transformer` during runtime, so don't worry about setting them now
- `loadSavedModel` accepts two params, first is the path to the exported model. The second is the SparkSession that is `spark` variable we previously started via `sparknlp.start()`
- NOTE: `loadSavedModel` accepts local paths in addition to distributed file systems such as `HDFS`, `S3`, `DBFS`, etc. This feature was introduced in Spark NLP 4.2.2 release. Keep in mind the best and recommended way to move/share/reuse Spark NLP models is to use `write.save` so you can use `.load()` from any file systems natively.st and recommended way to move/share/reuse Spark NLP models is to use `write.save` so you can use `.load()` from any file systems natively.

In [4]:
MODEL_NAME = "openai-community/gpt2"
EXPORT_PATH = f"ov_models/{MODEL_NAME}"

In [5]:
from sparknlp.annotator import GPT2Transformer

gpt2 = GPT2Transformer.loadSavedModel(EXPORT_PATH, spark)\
  .setInputCols(["documents"])\
  .setMaxOutputLength(50)\
  .setDoSample(True)\
  .setTopK(50)\
  .setTemperature(0)\
  .setBatchSize(5)\
  .setNoRepeatNgramSize(3)\
  .setOutputCol("generation")

Let's save it on disk so it is easier to be moved around and also be used later via `.load` function

In [6]:
gpt2.write().overwrite().save(f"{MODEL_NAME}_spark_nlp")

Let's clean up stuff we don't need anymore

In [7]:
!rm -rf {EXPORT_PATH}

Awesome  😎 !

This is your Openvino GPT2 model from HuggingFace 🤗  loaded and saved by Spark NLP 🚀

In [8]:
! ls -l {MODEL_NAME}_spark_nlp

total 486652
drwxr-xr-x 4 root root      4096 Jul  3 09:49 fields
-rw-r--r-- 1 root root 498315840 Jul  3 09:49 gpt2_openvino
drwxr-xr-x 2 root root      4096 Jul  3 09:49 metadata


Now let's see how we can use it on other machines, clusters, or any place you wish to use your new and shiny GPT2 model 😊

In [9]:
from sparknlp.base import DocumentAssembler
from sparknlp.annotator import GPT2Transformer
from pyspark.ml import Pipeline

test_data = spark.createDataFrame([
    ["Transfer learning, where a model is first pre-trained on a data-rich task before being fine-tuned on a " +
     "downstream task, has emerged as a powerful technique in natural language processing (NLP). The effectiveness " +
     "of transfer learning has given rise to a diversity of approaches, methodology, and practice. In this " +
     "paper, we explore the landscape of transfer learning techniques for NLP by introducing a unified framework " +
     "that converts all text-based language problems into a text-to-text format. Our systematic study compares " +
     "pre-training objectives, architectures, unlabeled datasets, transfer approaches, and other factors on dozens " +
     "of language understanding tasks. By combining the insights from our exploration with scale and our new " +
     "Colossal Clean Crawled Corpus, we achieve state-of-the-art results on many benchmarks covering " +
     "summarization, question answering, text classification, and more. To facilitate future work on transfer " +
     "learning for NLP, we release our dataset, pre-trained models, and code."]
]).toDF("text")

document_assembler = DocumentAssembler() \
    .setInputCol("text") \
    .setOutputCol("document")

gpt2 = GPT2Transformer.load(f"{MODEL_NAME}_spark_nlp") \
    .setInputCols(["document"]) \
    .setOutputCol("generation") \
    .setMaxOutputLength(50) \
    .setDoSample(True) \
    .setTopK(50) \
    .setTemperature(0) \
    .setBatchSize(5) \
    .setNoRepeatNgramSize(3)

pipeline = Pipeline().setStages([
    document_assembler,
    gpt2
])

model = pipeline.fit(test_data)
result = model.transform(test_data)

result.select("generation.result").show(truncate=False)

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

That's it! You can now go wild and use hundreds of GPT2 models from HuggingFace 🤗 in Spark NLP 🚀
